In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Load FashionMNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:01<00:00, 18.0MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 314kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 4.99MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 14.3MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
%pip install minisom




  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.3-py3-none-any.whl size=11706 sha256=b5c6c02d0889324ef671bc8b2226516d0d6c765a04b4502a623e986149b461b2
  Stored in directory: /root/.cache/pip/wheels/96/98/a5/52dee3e8ed1dbfc4d77e4da41b6d89dd7ab9ead1b921e766f8
Successfully built minisom


In [6]:
from minisom import MiniSom
import numpy as np

# Initialize SOM
som = MiniSom(x=10, y=10, input_len=784, sigma=1.0, learning_rate=0.5)

# Train SOM with flattened images
train_data = train_dataset.data.view(-1, 784).numpy()
som.train_random(data=train_data, num_iteration=100)

# Get SOM representations
som_transformed = np.array([som.winner(x) for x in train_data])


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class RBM(nn.Module):
    def __init__(self, visible_dim, hidden_dim):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(hidden_dim, visible_dim) * 0.1)
        self.h_bias = nn.Parameter(torch.zeros(hidden_dim))
        self.v_bias = nn.Parameter(torch.zeros(visible_dim))

    def forward(self, v):
        h_prob = torch.sigmoid(F.linear(v, self.W, self.h_bias))
        h_sample = torch.bernoulli(h_prob)
        return h_sample

rbm = RBM(784, 128)
# Get RBM representations
train_data_tensor = torch.tensor(train_data).float()
rbm_output = rbm(train_data_tensor).detach().numpy()


In [14]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

vae = VAE(784, 256, 64)

def vae_loss(recon_x, x, mu, logvar):
    # Mean Squared Error Loss
    MSE = F.mse_loss(recon_x, x, reduction='sum')
    # KL Divergence Loss remains the same
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
for epoch in range(10):
    for data, _ in train_loader:
        data = data.view(-1, 784)
        recon_batch, mu, logvar = vae(data)
        loss = vae_loss(recon_batch, data, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Get VAE representations
vae_output = vae(train_data_tensor)[0].detach().numpy()


In [18]:
from sklearn.impute import SimpleImputer

def evaluate_model(X_train, y_train, X_test, y_test):

    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    # Initialize classifiers
    rf = RandomForestClassifier()
    ab = AdaBoostClassifier(algorithm='SAMME')

    # Train and evaluate RandomForest
    rf.fit(X_train, y_train)
    rf_acc = accuracy_score(y_test, rf.predict(X_test))

    # Train and evaluate AdaBoost
    ab.fit(X_train, y_train)
    ab_acc = accuracy_score(y_test, ab.predict(X_test))

    return rf_acc, ab_acc

# Original Data
rf_acc_orig, ab_acc_orig = evaluate_model(train_data, y_train, test_data, y_test)

# SOM Data
rf_acc_som, ab_acc_som = evaluate_model(som_transformed, y_train, som_test_transformed, y_test)

# RBM Data
rf_acc_rbm, ab_acc_rbm = evaluate_model(rbm_output, y_train, rbm_test_output, y_test)

# VAE Data
rf_acc_vae, ab_acc_vae = evaluate_model(vae_output, y_train, vae_test_output, y_test)

# Print results
print(f"Original RF: {rf_acc_orig:.4f}, AdaBoost: {ab_acc_orig:.4f}")
print(f"SOM RF: {rf_acc_som:.4f}, AdaBoost: {ab_acc_som:.4f}")
print(f"RBM RF: {rf_acc_rbm:.4f}, AdaBoost: {ab_acc_rbm:.4f}")
print(f"VAE RF: {rf_acc_vae:.4f}, AdaBoost: {ab_acc_vae:.4f}")


Original RF: 0.8770, AdaBoost: 0.5089
SOM RF: 0.5467, AdaBoost: 0.3782
RBM RF: 0.8070, AdaBoost: 0.4637
VAE RF: 0.3234, AdaBoost: 0.2735
